In [ ]:
import numpy as np
import onnxruntime as rt

# Load the model
new_session = rt.InferenceSession("model/bad_model.onnx")

def get_predict_label_and_confidence(session, solution):
    y_pred_solution = session.run(None, {'X': solution.astype(np.float32)})
    print(y_pred_solution)
    # Predict checked or not checked
    label = y_pred_solution[0][0]
    
    # Only have two classes: checked and not checked
    confidence_top1_label = y_pred_solution[1][0][0]
    confidence_top2_label = y_pred_solution[1][0][1]
    
    return (label, confidence_top1_label, confidence_top2_label)


def get_fitness(y_true, y_pred): 
    # Get the predicted label and confidence
    label_original = y_true[0]
    label_mutated, confidence_top1_label, confidence_top2_label = y_pred
    
    # If the prediction is different from the true label: fitness = - confidence_top1_label
    # If the prediction is the same as the true label: fitness = confidence_top1_label - confidence_top2_label    
    return -confidence_top1_label if label_mutated != label_original else confidence_top1_label - confidence_top2_label


def random_gaussian_mutation(solution, mutation_rate=0.5, mutation_std=0.25):
    mutated_solution = solution.copy()
    num_features = len(mutated_solution)
    num_mutations = max(1, int(mutation_rate * num_features))
    
    for _ in range(num_mutations):
        feature_index = np.random.randint(num_features)
        mutated_solution[0, feature_index] += np.random.normal(scale=mutation_std)
        
    return mutated_solution


def stochastic_hill_climber(initial_solution, session, y_test, num_iterations=1000, num_neighbors=100):
    current_solution = initial_solution
    current_fitness = get_fitness(y_test, get_predict_label_and_confidence(session, current_solution))
    
    for _ in range(num_iterations):
        neighbors = [random_gaussian_mutation(current_solution) for _ in range(num_neighbors)]
        neighbor_scores = [get_fitness(y_test, get_predict_label_and_confidence(session, neighbor)) for neighbor in neighbors]
        
        # Minimize the objective function
        best_neighbor_index = np.argmin(neighbor_scores)
        best_neighbor_score = neighbor_scores[best_neighbor_index]
        
        # Check if the best neighbor is better than the current solution
        if best_neighbor_score < current_fitness:
            current_solution = neighbors[best_neighbor_index]
            current_fitness = best_neighbor_score
    
    return current_solution, current_fitness


# Get a random sample from the test set
rand_sample_index = np.random.randint(0, len(X_test)-1)
initial_solution = X_test[rand_sample_index:rand_sample_index+1].values

# Run the algorithm
final_solution, best_score = stochastic_hill_climber(initial_solution, new_session, y_test[0:1].values)

print("Best solution:", final_solution)
print("Best O score:", best_score)
